In [1]:
import folium
import pandas as pd

coordinative_csv = "../output_csv/bremen_data_v5_2_with_coordinate.csv"
df_artist_with_coordinate = pd.read_csv(coordinative_csv) 
df_artist_with_coordinate["birth year"] = df_artist_with_coordinate["birth year"].astype("Int64")
df_artist_with_coordinate["birth_year_span_start"] = df_artist_with_coordinate["birth_year_span_start"].astype("Int64")

df_artist_with_coordinate.head()


,artistGNDId,artistName (preferred),placeOfBirth,birth year,num_works,birth_year_span_start,latitude,longitude
0,100076262,"Caylus, Anne Claude Philippe de",Paris,1692,24,1650,48.858890,2.320041
1,100104401,"Dequevauviller, François",Abbeville,1745,1,1700,50.106083,1.833703
2,100648916,"Collas, Achille",Paris,1795,3,1750,48.858890,2.320041
3,100786294,"Porret, Henri Désiré",Lille,1800,6,1800,50.636565,3.063528
4,100811833,"Kraus, Johann Ulrich",Augsburg,1655,48,1650,48.369034,10.897952


In [2]:
df_artist_with_coordinate["placeOfBirth"].value_counts()


placeOfBirth
Paris                                         278
Nürnberg                                     115
Antwerpen                                      56
Amsterdam                                      40
Augsburg                                       31
                                             ... 
Sedan                                           1
Rive-de-Gier                                    1
Vernon <Eure>                                   1
Maidstone                                       1
Toul (Dép.Meurthe-et-Moselle) (od. Metz?)      1
Name: count, Length: 402, dtype: int64

In [3]:
df_artist_with_coordinate["birth_year_span_start"].value_counts()

birth_year_span_start
1800    308
1750    192
1500    189
1700    176
1600    159
1550    119
1850     91
1650     79
1450     68
1400     15
4950      2
1900      2
Name: count, dtype: Int64

Split dataframe to multiple dataframes based on `birth_year_span_start`

In [4]:
grouped_dfs = {
    year: group for year, group in df_artist_with_coordinate.groupby("birth_year_span_start")
}
print(grouped_dfs.keys())
grouped_dfs[1400].head()

dict_keys([np.int64(1400), np.int64(1450), np.int64(1500), np.int64(1550), np.int64(1600), np.int64(1650), np.int64(1700), np.int64(1750), np.int64(1800), np.int64(1850), np.int64(1900), np.int64(4950)])


,artistGNDId,artistName (preferred),placeOfBirth,birth year,num_works,birth_year_span_start,latitude,longitude
35,102359229,"Anonym, deutsch, 15. Jahrhundert",Straßburg,1401,3,1400,48.584614,7.750713
189,1065633092,Zoan Andrea,Mantua,1424,6,1400,45.169263,10.670837
445,11852786X,"Anonym, deutsch, 15. Jahrhundert",Nürnberg,1401,5,1400,49.453872,11.077298
563,118563890,"Anonym, deutsch, 15. Jahrhundert",Nürnberg,1401,5,1400,49.453872,11.077298
578,118577336,"Mantegna, Andrea",Padua,1431,1,1400,45.407717,11.873446


In [5]:
merged_dfs_by_year = {}

for year, group in grouped_dfs.items():
    merged_df = group.groupby("placeOfBirth").agg({
        "artistName (preferred)": lambda x: "; ".join(x.dropna().unique()),
        "placeOfBirth": "count",  # count how many artists per place
        "latitude": "first",
        "longitude": "first"
    }).rename(columns={
        "artistName (preferred)": "artists",
        "placeOfBirth": "artistCount"
    }).reset_index()
    
    merged_dfs_by_year[year] = merged_df

merged_dfs_by_year[1400].head()

,placeOfBirth,artists,artistCount,latitude,longitude
0,Colmar,"Anonym; Anonym, deutsch; Anonym, deutsch, 15. ...",5,48.077752,7.357964
1,Koblenz,"Anonym, französisch, 15. Jahrhundert",1,50.353328,7.594395
2,Mantua,Zoan Andrea,1,45.169263,10.670837
3,Markgröningen,"Anonym, deutsch, 15. Jahrhundert",1,48.905063,9.080811
4,Nürnberg,"Anonym, deutsch, 15. Jahrhundert; Wolgemut, Mi...",3,49.453872,11.077298


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from branca.element import Template, MacroElement

year_slider = widgets.IntSlider(
    value=min(merged_dfs_by_year.keys()),
    min=1450,
    max=1900,
    step=50, 
    description='Year:'
)

def update_map(year):
    clear_output(wait=True)
    display(year_slider)
    
    df = merged_dfs_by_year[year]

    # Create a new map
    m = folium.Map(location=[50, 10], zoom_start=4)

    # Color based on artist count
    for _, row in df.dropna(subset=["latitude", "longitude"]).iterrows():
        count = row["artistCount"]
        if count > 10:
            color = "darkred"
        elif count >= 5:
            color = "orange"
        elif count >=  3:
            color = "blue"
        else:
            color = "green"

        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            # radius=5 + count * 0.3,
            radius= 6,
            popup=folium.Popup(f"<b>Place of Birth:</b> {row['placeOfBirth']}<br> <b>Artists:</b> {row['artists']}<br><b>Count:</b> {row['artistCount']}", max_width=300),
            color=color,
            fill=True,
            fill_opacity=0.7
        ).add_to(m)
    
    legend_html = """
    {% macro html(this, kwargs) %}
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        width: 150px;
        height: 120px;
        background-color: white;
        border:2px solid grey;
        z-index:9999;
        font-size:14px;
        padding: 10px;
        ">
        <b>Artist Count</b><br>
        <i style="color:darkred;">●</i> > 10<br>
        <i style="color:orange;">●</i> 5–10<br>
        <i style="color:blue;">●</i> 3–4<br>
        <i style="color:green;">●</i> 1–2<br>
    </div>
    {% endmacro %}
    """
    legend = MacroElement()
    legend._template = Template(legend_html)
    m.get_root().add_child(legend)

    display(m)

# 3. Bind slider to map update
year_slider.observe(lambda change: update_map(change['new']), names='value')

# 4. Display the slider and the first map
display(year_slider)
update_map(year_slider.value)

IntSlider(value=1600, description='Year:', max=1900, min=1450, step=50)